## Do These Things With the Data

Once you have your csv files on your computer, do the following:

1) Import each of the csv files you downloaded from the SSCC into a pandas DataFrame.

In [11]:
import pandas as pd

# For the data files (mail, item and customer), read the CSVs into a dataframe. Use 
# an empty string to fill in missing data.  Assumes data is in local path

df_mail = pd.read_csv("mail.csv", sep = ",")
df_mail = df_mail.fillna('')
print df_mail.head(5),"\n"
df_item = pd.read_csv("items.csv", sep = ",")
df_item = df_item.fillna('')
df_cust = pd.read_csv("customers.csv", sep = ",")
df_cust = df_cust.fillna('')

   acctno  mail_1  mail_2  mail_3  mail_4  mail_5  mail_6  mail_7  mail_8  \
0  WLPAQS       0       0       0       1       0       1       1       1   
1  WGDQLA       0       0       0       0       0       0       0       0   
2  WDPSHS       1       0       0       0       0       0       1       0   
3  APSYYW       1       1       1       1       1       1       1       1   
4  SDHLPH       1       1       1       1       1       1       1       1   

   mail_9  mail_10  mail_11  mail_12  mail_13  mail_14  mail_15  mail_16  
0       1        1        1        0        1        0        1        0  
1       0        0        0        1        1        0        0        1  
2       0        1        0        0        1        0        1        0  
3       1        1        1        1        1        1        1        1  
4       1        1        1        1        1        1        1        1   



2) Verify that there are no duplicate customer records in the _customer_ data.

In [12]:
# Count the number of duplicate customer records, and report it
# where "duplicate" means all field (column) values for any 2 rows are the same
dup_cust = df_cust.duplicated().sum()
print "There were", dup_cust, "duplicate Customer records found\n"

There were 0 duplicate Customer records found



3) Check the item and mail data to determine whether there are any records in them for customers who are _not_ in the customer data.

In [13]:
# create a list of the unique customer account numbers
cust_acc = list(df_cust['acctno'].unique())

# Since we are looking for records that are in df_mail, but NOT in df_cust, use the 
#  pd.isin() function, and set to '==False' for records where the value of 
# df_mail.acctno is not in the list of unique Customer acctno's.  Repeat for df_item.  
missing_mail = df_mail[df_mail.acctno.isin(cust_acc)==False]
print "There are ", len(missing_mail)," account numbers in Mail not in Customers"
missing_item = df_item[df_item.acctno.isin(cust_acc)==False]
print "There are ", len(missing_item)," account numbers in Item not in Customers"

There are  0  account numbers in Mail not in Customers
There are  0  account numbers in Item not in Customers


4) Create a __sqlite__ database, and write the customer, item, and mail data into it as tables.  _Do not include any item or mail data that don't match a customer_.  You may want to use the sqlalchemy Python package to do this, or the sqlite3 Python package.  Save your sqlite database for use in the future. (You may be asked to share it.)

In [15]:
# filter the dataframes to remove items that are in mail or item and not in customer, 
# save cleaned dataframe to a new name for clarity, and for importing into the DB
clean_mail = df_mail[df_mail.acctno.isin(missing_mail['acctno'])==False]
clean_item = df_item[df_item.acctno.isin(missing_item['acctno'])==False]

# Check for database, and if it exists, delete it so can be recreated clean
import os
import errno
try:
    os.remove('XYZ_DB.db')
except OSError as e:
    if (e == 2):
        pass

# Create the SQLite3 database by instantiating a connection and importing the "clean"
# dataframes. Commit and close to "flush" to disk
import sqlite3
db_conn = sqlite3.connect('XYZ_DB.db')
clean_mail.to_sql("mail", db_conn, if_exists='replace')
clean_item.to_sql("item", db_conn, if_exists="replace")
df_cust.to_sql("customers", db_conn, if_exists='replace')
db_conn.commit()
db_conn.close()

# verify 1) there are data in the tables,  2) the dataframe length matches the number 
# of rows in the table and 3) that a random row is the same between the dataframe and 
# table adjusting for index differences. 
# This should work since order is preserved on import
db_conn = sqlite3.connect('XYZ_DB.db')
c = db_conn.cursor()

def check_table(df, str_table):
    import random
    tmp_var = '--tempVar--'
    r_count = 0
    q1 = 'select * from tmp_var'.replace('tmp_var', str_table)
    for row in c.execute (q1):
        r_count +=1
    i = random.randint(1, len(df))
    q2 = 'select rowid, * from tmp_var where rowid = ?'.replace('tmp_var', str_table)
    q_cursor = c.execute(q2, [i])
    s_res = q_cursor.fetchone()
    d_res = df.ix[i-1]
    if (s_res[2] == d_res.acctno) and (r_count == len(df)):
        res_str = "Length and random row match, tmp_var table\
            is good".replace('tmp_var', str_table)
        print res_str
    else:
        res_str = """Length and random row match, tmp_var table don't match, 
            there was an error""". replace('tmp_var', str_table)
        print res_str
    
print "******* Verifying Tables ********"
check_table(clean_mail, 'mail')
check_table(clean_item, 'item')
check_table(df_cust,'customers')

******* Verifying Tables ********
Length and random row match, mail table            is good
Length and random row match, item table            is good
Length and random row match, customers table            is good


5) Create and export a csv file for XYZ to target in a direct mail marketing campaign.  This file should have the following data fields in it, and it should have a header record giving field names.  It should only include customers who have been mailed at least seven(7) times in XYZ's 16 mail campaigns.

The first four "Z" variables above are from Experian.  They have categorical value codes that you'll find defined in the XYZ Company data documentation on Canvas "Resources." For each of these "Z" variables, create a version that is numeric and is coded "1" (numeric) if the original is coded "Y," and "0" (numeric) for any other original code.

In [16]:
# create a new table called "mailed" with the customers who have been 
# mailed 7 or more times
c.execute("""
    create table if not exists mailed as 
    select acctno, 
    (mail_1+mail_2+mail_3+mail_4+mail_5+mail_6+mail_7+mail_8+mail_9+
    mail_10+mail_11+mail_12+mail_13+mail_14+mail_15+mail_16) as total_mailed 
    from mail
    where total_mailed >= 7;""")

# create the table for people to mail new campaign to; call new table "target". 
# Table "target" matches acctno from the "customer" and "mailed" tables to form 
# a new collection columns of interest for the campaign
c.execute("""
    create table if not exists target as 
    select mailed.acctno, total_mailed, YTD_TRANSACTIONS_2009, YTD_SALES_2009,
      ZHOMEENT, ZMOBAV, ZCREDIT, ZHITECH 
    from mailed, customers
    where mailed.acctno = customers.acctno;""")

# add the new columns for coding into, then perform the coding.  
# My understanding of the task is that an empty field gets no code in the new columns.
c.execute("""alter table target add column ZHOMEENT01 int;""") 
c.execute("""alter table target add column ZMOBAV01 int;""")
c.execute("""alter table target add column ZCREDIT01 int;""")
c.execute("""alter table target add column ZHITECH01 int;""")
c.execute("""update target set ZHOMEENT01 = (case when ZHOMEENT = "Y" then 1 when 
    (ZHOMEENT <> "Y" and (ZHOMEENT <> "" or ZHOMEENT <> null)) then 0 end);""")
c.execute("""update target set ZMOBAV01 = (case when ZMOBAV = "Y" then 1 when 
    (ZMOBAV <> "Y" and (ZMOBAV <> "" or ZMOBAV <> null)) then 0 end);""")
c.execute("""update target set ZCREDIT01 = (case when ZCREDIT = "Y" then 1 when 
    (ZCREDIT <> "Y" and (ZCREDIT <> "" or ZCREDIT <> null)) then 0 end);""")
c.execute("""update target set ZHITECH01 = (case when ZHITECH = "Y" then 1 when 
    (ZHITECH <> "Y" and (ZHITECH <> "" or ZHITECH <> null)) then 0 end);""")

#visually inspect for correctness; 
# 6 page limit precludes getting fancy with verification
print "****** Visually inspect coding results ****"
print "Correct: 'Y' -> 1, blank or null -> None, all else -> 0\n"
for row in c.execute("select * from target limit 12"):
    print row
print '\n'

#write the target table as a CSV file
write_table = pd.read_sql_query('select * from target',db_conn)
write_table.to_csv('target.csv', header=True, index=False)


****** Visually inspect coding results ****
Correct: 'Y' -> 1, blank or null -> None, all else -> 0

(u'WLPAQS', 9, 0, 0, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'APSYYW', 16, 1, 129, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'SDHLPH', 16, 1, 477, u'Y', u'U', u'Y', u'Y', 1, 0, 1, 1)
(u'DLWWDP', 12, 3, 1806, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'DGDSAD', 16, 7, 2229, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'WWLYGYA', 15, 0, 0, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'WWLYGYA', 15, 0, 0, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'WWGGQHS', 16, 10, 9285, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'WAWYWSQ', 8, 0, 0, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)
(u'WLGLADQ', 14, 0, 0, u'Y', u'Y', u'Y', u'Y', 1, 1, 1, 1)
(u'AQAPAHW', 14, 2, 186, u'', u'', u'', u'', None, None, None, None)
(u'AQSLWWW', 15, 1, 144, u'U', u'U', u'Y', u'Y', 0, 0, 1, 1)




6) Using the data that you created for exporting to the csv file, create a cross-tabulation table for each of the Z variables and the numeric versions of them that you created, showing for each that that your recoding worked as required.  Understand that the kind of cross-tabulation required has one variable's values in the rows, and the other variable's values in the columns.  Be sure to consider missing values. The total number of cases shown in your crosstabs should equal the number of customers in the csv file. Be careful to take any missing values into account in your table.)

In [17]:
# do the cross-tabs for each "Z_Something"-pair, print as we go
print pd.crosstab(write_table.zhomeent, write_table.ZHOMEENT01,margins = True), '\n'
print pd.crosstab(write_table.zmobav, write_table.ZMOBAV01,margins = True), '\n'
print pd.crosstab(write_table.zcredit, write_table.ZCREDIT01,margins = True), '\n'
print pd.crosstab(write_table.zhitech, write_table.ZHITECH01,margins = True), '\n'


ZHOMEENT01   0.0   1.0   All
zhomeent                    
U           6384     0  6384
Y              0  1448  1448
All         6384  1448  7832 

ZMOBAV01   0.0  1.0   All
zmobav                   
U         7593    0  7593
Y            0  239   239
All       7593  239  7832 

ZCREDIT01  0.0   1.0   All
zcredit                   
U          270     0   270
Y            0  7562  7562
All        270  7562  7832 

ZHITECH01  0.0   1.0   All
zhitech                   
U          519     0   519
Y            0  7313  7313
All        519  7313  7832 



7) Pickle or Shelve your pandas DataFrames. Save your sqlite DB.

In [18]:
#Commit the changes to the DB and close the connection to save to local folder
db_conn.commit()
db_conn.close()

# pickle mail, item and customer dataframes, stored in local working directory
clean_mail.to_pickle('XYZ_mail.pkl')
clean_item.to_pickle('XYZ_item.pkl')
df_cust.to_pickle('XYZ_cust.pkl')